## Домашнее задание 3

Измерены следующие признаки:

- state — штат США
- account_length — длительность использования аккаунта
- area_code — деление пользователей на псевдорегионы, использующееся в телекоме
- intl_plan — подключена ли у пользователя услуга международного общения
- vmail_plan — подключена ли у пользователя услуга голосовых сообщений
- vmail_message — количество голосых сообщений, который пользователь отправил / принял
- day_calls — сколько пользователь совершил дневных звонков
- day_mins — сколько пользователь проговорил минут в течение дня
- day_charge — сколько пользователь заплатил за свою дневную активность
- eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности
- night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности
- intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения
- custserv_calls — сколько раз пользователь позвонил в службу поддержки
- treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)
- mes_estim — оценка интенсивности пользования интернет мессенджерами
- churn — результат оттока: перестал ли абонент пользоваться услугами оператора

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.stats import chi2_contingency,fisher_exact
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('churn_analysis.csv', index_col = 0)
data.head()

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


# Гипотеза о влиянии штата

Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора. Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn. Заметьте, что, например, (AZ, HI) и (HI, AZ) — это одна и та же пара. Какой критерий подходит для решения этой задачи? Сколько достигаемых уровней значимости оказались меньше, чем α=0.05?

In [3]:
data1 = data[data['treatment']==1]
data1.head()

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
8,LA,117,408,no,no,0,184.5,97,31.37,351.6,...,215.8,90,9.71,8.7,4,2.35,1,1,0.50,False.
12,IA,168,408,no,no,0,128.8,96,21.90,104.9,...,141.1,128,6.35,11.2,2,3.02,1,1,0.37,False.
17,VT,93,510,no,no,0,190.7,114,32.42,218.2,...,129.6,121,5.83,8.1,3,2.19,3,1,0.84,False.


Преобразуем для удобства признак churn: 0 =  False. и 1 = True.

In [4]:
data1['churn'].replace(['True.', 'False.'], [1, 0], inplace = True)

/Users/annatihonova/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:3430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [5]:
state_churn = pd.DataFrame()
state_churn['state'] = data1.state
state_churn['churn'] = data1.churn
state_churn.head()

,state,churn
0,KS,0
3,OH,0
8,LA,0
12,IA,0
17,VT,0


Столько различных значений принимает state:

In [6]:
len(state_churn['state'].unique())

51

Посмотрим на эти значения:

In [7]:
state_churn['state'].unique()

array(['KS', 'OH', 'LA', 'IA', 'VT', 'CO', 'WY', 'MO', 'GA', 'AK', 'MA',
       'NJ', 'AR', 'MI', 'IN', 'MN', 'SD', 'TX', 'DC', 'NY', 'OR', 'AZ',
       'MS', 'UT', 'ME', 'ID', 'FL', 'NV', 'WA', 'WV', 'NH', 'VA', 'NM',
       'KY', 'MT', 'PA', 'RI', 'DE', 'OK', 'NE', 'CT', 'WI', 'CA', 'NC',
       'MD', 'TN', 'AL', 'IL', 'ND', 'HI', 'SC'], dtype=object)

In [13]:
states = state_churn['state'].unique()

Пары каждого уникального значения с каждым:

In [14]:
pairs = []
for i in range(len(state_churn['state'].unique())):
    for j in range(i+1,len(state_churn['state'].unique())):
        s_i = states[i]
        s_j = states[j]
        pairs.append([s_i, s_j])

Всего таких пар:

In [15]:
len(pairs)

1275

In [16]:
cont_tables=[]
for i in range(len(state_churn['state'].unique())):
    for j in range(i+1,len(state_churn['state'].unique())):
        s_i = states[i]
        s_j = states[j]
        tables = pd.concat([state_churn[state_churn.state == s_i], state_churn[state_churn.state == s_j]], 0)
        tables['churn_1'] = state_churn.churn
        tables['churn_0'] = abs(1- tables['churn_1'])
        tables=tables.drop(['churn'], axis=1)
        cont_tables.append(tables.groupby(tables.state).sum().unstack())

In [21]:
cont_tables[:10]

[         state
 churn_1  KS        7
          OH        2
 churn_0  KS       18
          OH       22
 dtype: int64,          state
 churn_1  KS        7
          LA        0
 churn_0  KS       18
          LA       15
 dtype: int64,          state
 churn_1  IA        2
          KS        7
 churn_0  IA       13
          KS       18
 dtype: int64,          state
 churn_1  KS        7
          VT        5
 churn_0  KS       18
          VT       18
 dtype: int64,          state
 churn_1  CO        5
          KS        7
 churn_0  CO       17
          KS       18
 dtype: int64,          state
 churn_1  KS        7
          WY        2
 churn_0  KS       18
          WY       20
 dtype: int64,          state
 churn_1  KS        7
          MO        4
 churn_0  KS       18
          MO       17
 dtype: int64,          state
 churn_1  GA        2
          KS        7
 churn_0  GA       19
          KS       18
 dtype: int64,          state
 churn_1  AK        1
          KS      

Используем Фишера. Найдем p-value:

In [22]:
p = []
for i in cont_tables:
    c= np.array(i).reshape(2,2)
    p_i= fisher_exact(c, alternative='two-sided')[1]
    p.append(p_i)

Гипотеза о независимости state и churn не будет отвергаться при всех уровнях значимости, которые меньше p-value. Cравним найденные для пар p-value c уровнем значимости 0.05 и посмотрим, если ли пары для которых основная гипотела не отвергается:

In [23]:
bad_p=[]
for i in p:
    if i < 0.05:
        bad_p.append(i)
len(bad_p)

10

Не все найденные p-value больше 0.05. Гипотезы о независимости отвергаются в 10 парах штатов при уровне значимости  5%. Учитывая, что всего пар 1275, а гипотеза отвергается только в 10 из них, можно говорить о том, что связь между признаками отсутствует и ,следовательно, отток клиентов не зависит от штата.

# Корреляции Пирсона и Спирмена

Посчитайте корреляции Пирсона и Спирмена между day_calls и mes_estim на всех данных, оцените их значимость, дайте интерпретацию результата

In [24]:
from scipy.stats import pearsonr, spearmanr

In [109]:
pearson_corr= pearsonr(data['day_calls'], data['mes_estim'])[0]
print('Pearson:',pearson_corr)
spearman_corr= spearmanr(data['day_calls'], data['mes_estim'])[0]
print('Spearman:',spearman_corr)

('Pearson:', -0.051794350587572625)
('Spearman:', 0.043349880533927444)


Значимость корреляций:

In [110]:
pearson_p= pearsonr(data['day_calls'], data['mes_estim'])[1]
print('P-value for Pearson: ',pearson_p)
spearman_p= spearmanr(data['day_calls'], data['mes_estim'])[1]
print('P-value for Spearman: ',spearman_p)

('P-value for Pearson: ', 0.0027798836869756707)
('P-value for Spearman: ', 0.012317367189170541)


- Гипотеза о том, что корреляция Пирсона равная нулю отвергается при минимальном уровне значимости 0.0028 (т.е.уровнях значимости <0.2%) -->  корреляция отсутствует
- Гипотеза о том, что корреляция Спирмана равная нулю отвергается при минимальном уровне значиости 0.012 (т.е. при уровнях значимости <1.2%) --> корреляция отсутствует



- Можно сказать, что корреляция Пирсона значима при всех разумных уровнях значимости
- А корреляция Спирмана не значима при уровне значимости 1%, но при уровнях 5% и 10% найденная корреляция значима.

# Корреляция Крамера

Посчитайте значение коэффицента корреляции Крамера между штатом (state) и оттоком пользователей (churn) для всех пользователей, которые находились в контрольной группе (treatment=1). Проверьте гипотезу об отсутствии связи между этими признаками.

Воспользуемся таблицей, созданной ранее, включающей: контрольную группу, признаки state и churn

In [28]:
state_churn.head()

,state,churn
0,KS,0
3,OH,0
8,LA,0
12,IA,0
17,VT,0


Построим таблицу более удобным для данной задачи образом :

In [29]:
data3 = data[data['treatment'] == 1][['state', 'churn']]
c_matrix2 = pd.crosstab(data3['state'], data3['churn'])
c_matrix2.head()

churn,False.,True.
state,,
AK,19,1
AL,25,5
AR,11,5
AZ,17,2
CA,10,5


Найдем сколько значений для True. и False. :

In [30]:
n = c_matrix2.sum()
n

churn
False.    917
True.     180
dtype: int64

In [111]:
chi2= chi2_contingency(c_matrix2)[0]
V= np.sqrt(chi2 /((917+180)*(min(2,1275)-1)))
print('Kramer: ',V)

('Kramer: ', 0.20039321502033319)


Коэффициент Крамера принимает значение 1 при сильной связи и значение 0 при отсутствии связи. Следовательно, чтобы проверить значение связи между признаками, необходимо проверить гипотезу, что коэффициент Крамера=0. 

Коэффициент Крамера = 0, когда числитель равен 0, поэтому: 

In [32]:
p_val = chi2_contingency(c_matrix2, correction=False)[1]
p_val

0.70975900427784411

Значение p_value = 71% достаточно большое --> на всех разумных уровнях значимости гипотеза об отсутствии связи между признаками не отвергается.
Значит найденная связь между признаками статистически не значима

# Анализ эффективности методов удержания

Проведите анализ эффективности удержания (churn) с помощью раличных методов (treatment = 0, treatment = 2) относительно контрольной группы пользователей (treatment = 1). Что можно сказать об этих двух методах (treatment = 0, treatment = 2)? Одинаковы ли они с точки зрения эффективности? Каким бы методом вы бы посоветовали воспользоваться компании?

Эффективность удержания это вероятность того что человек перестанет пользоваться услугами при условии определенного значения treatment. Определим эту вероятность для различных значений treatment:

In [92]:
from __future__ import division

In [113]:
data1 = data[data['treatment']==1]
n_churn_1 = data1[data1['churn']=='True.']
k_1 = n_churn_1.churn.count()
n_1= data1.churn.count()
print('Probability that person leave (if he/she is from treatment=1 group):',k_1/n_1 )

('Probability that person leave (if he/she is from treatment=1 group):', 0.16408386508659981)


In [115]:
data0 = data[data['treatment']==0]
n_churn_0 = data0[data0['churn']=='True.'] 
k_0 = n_churn_0.churn.count()
n_0= data0.churn.count() 
print('Probability that person leave (if he/she is from treatment=0 group)',k_0/n_0 )

('Probability that person leave (if he/she is from treatment=0 group)', 0.14563106796116504)


In [116]:
data2 = data[data['treatment']==2]
n_churn_2 = data2[data2['churn']=='True.'] 
k_2 = n_churn_2.churn.count()
n_2= data2.churn.count() 
print('Probability that person leave (if he/she is from treatment=2 group)',k_2/n_2 )

('Probability that person leave (if he/she is from treatment=2 group)', 0.12511332728921123)


Наименьшая вероятность у группы treatment=2, значит можно сказать, что метод 2 наиболее эффективен. Оценим  значимость данного результата.

Используем хи-квадрат критерий. Составим таблицы сопряженности для treatment 1,2 и для treatment 0,1

Для treatment 0 и treatment 1:


In [101]:
fraim02= pd.DataFrame()
fraim02['prob_churn']= [k_0,k_1]
fraim02['prob_no_churn']= [(n_0-k_0),(n_1-k_1)]

In [102]:
fraim02

,prob_churn,prob_no_churn
0,165,968
1,180,917


In [103]:
chi_2= chi2_contingency(fraim02)[0]
chi_2

1.3135522978703889

In [104]:
p_val= chi2_contingency(fraim02)[1]
p_val

0.25175249831939217

При всех разумных уровнях значимости можно утверждать, что нет зависимости и что treatment=0 не более эффективен чем treatment=1 для удержания клиентов.


Для treatment 1 и treatment 2:


In [105]:
fraim12= pd.DataFrame()
fraim12['prob_churn']= [k_1,k_2]
fraim12['prob_no_churn']= [(n_1-k_1),(n_2-k_2)]

In [106]:
fraim12

,prob_churn,prob_no_churn
0,180,917
1,138,965


In [107]:
chi_2= chi2_contingency(fraim12)[0]
chi_2

6.4435991416199299

In [108]:
p_val= chi2_contingency(fraim12)[1]
p_val

0.011135281386020096

При всех уровнях значимости больше, чем 1.1%, можно утверждать, что зависимость есть и что treatment=2 действительно более эффективен, чем treatment=1


Принимая в качестве уровня значимости стандартные 5%, можно дать совет компании использовать метод для удержания клиентов treatment 2.